In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 5.1MB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5 
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/ml_matrix"

/content/drive/My Drive/Colab Notebooks/ml_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(features, model = DecisionTreeRegressor(max_depth=5)):
  X = df[features].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [7]:
df['brand_cat'] = df['brand'].map(lambda x:str(x).lower()).factorize()[0]

run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [8]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)


(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict

df['features_parsed'] = df['features'].map(parse_features) 

In [27]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [11]:
def get_name_feat(key):
  return 'feat_'+key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_sock style_cat', 'feat_product dimensions_cat', 'feat_style:_cat',
       'feat_guaranteed authentic_cat', 'feat_antiscratch lens coating_cat',
       'feat_handbag style_cat', 'feat_case back_cat', 'feat_shade_cat',
       'feat_leather :_cat', 'feat_target audience_cat'],
      dtype='object', length=1002)

In [13]:
{k:v for k,v in keys_stat.items() if v >30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
#df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_shape_cat'] = df['feat_shape'].factorize()[0]
#df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [0]:
feats_cat =  [x for x in df.columns if 'cat' in x]

In [0]:
feats=['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat', 'feat_shape_cat']
#feats += feats_cat
feats =   list(set(feats))
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [78]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)


(-57.146079902564885, 4.2669090536117755)


Weight,Feature
0.2605 ± 0.0060,brand_cat
0.1070 ± 0.0145,feat_material_cat
0.0248 ± 0.0022,feat_gender_cat
0.0220 ± 0.0017,feat_brand_cat
0.0134 ± 0.0008,feat_shape_cat
0.0039 ± 0.0009,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [18]:
df.brand.value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
pleaser shoes    0.000055
nurse mates      0.000055
zildjian         0.000055
korkers          0.000055
mr.chaos         0.000055
Name: brand, Length: 1732, dtype: float64

In [33]:
df[df['brand']=='nike'].features_paresd

AttributeError: ignored

0        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2        {'gender': 'men', 'color': 'black', 'shipping ...
3        {'gender': 'men', 'color': 'black', 'shipping ...
4        {'gender': 'men', 'color': 'black', 'shipping ...
                               ...                        
18275    {'style': 'desert boots', 'pattern': 'solid', ...
18276    {'shoe size': '9.5', 'gender': 'men', 'shoe ca...
18277    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18278    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18279    {'gender': 'unisex', 'frame style': 'wrap', 'p...
Name: features_parsed, Length: 18280, dtype: object

In [91]:
cd "content/drive/My Drive/Colab Notebooks/ml_matrix"

[Errno 2] No such file or directory: 'content/drive/My Drive/Colab Notebooks/ml_matrix'
/content/drive/My Drive/Colab Notebooks/ml_matrix


In [84]:
!git add ml_matrix/matrix_d3/day5.ipynb

fatal: pathspec 'ml_matrix/matrix_d3/day5.ipynb' did not match any files
